In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt 
import matplotlib
import re
from datetime import datetime
from wordcloud import WordCloud 
import jieba
import jieba.analyse
from datetime import date
import re
import string
from wordcloud import WordCloud 

### 構面字典

In [130]:
# dict_ = pd.read_csv('../構面字典-new.csv')
dict_ = pd.read_csv('./構面字典-3.csv')
df_dict = dict_[['name','alias']]
d_list = df_dict.to_dict(orient='records')
structure_dict = {} # 構面字典 => name:alias
for i in d_list:
    structure_dict[i['name']] = i['alias']
structure_dict.keys()#key:成就感,...

dict_keys(['成就感', '學習成長', '創新', '薪資', '福利', '管理制度', '工作氛圍', '同事互動', '主管風格', '工作地點', '公司規模', '工作環境', '產業前景', '輪調', '外派', '出差', '引擎', '馬力', '避震', '外觀', '操控', '安全', '堅固', '配備', '價錢', '科技', '品質', '折舊', '品牌', '空間', '保養', '續航', '尺寸', '車種', '驅動', '變速箱', '電資', '機械', '製造', '車輛工程', '品管', '多元性', '企業社會責任', '企業永續目標', '面試', '徵才', '實習', '工作', '離職', '轉職', '新鮮人', '畢業', '出路', '能力', '招募人員', '正向', '負向'])

In [131]:
structure_origin = dict_[['class','name']]
structures_list = structure_origin['class'].unique().tolist() # 有哪些構面字陣列

structure_origin_list = pd.DataFrame(structure_origin.groupby('class')['name'].apply(list)).reset_index().to_dict(orient='records')
transformed_data = [{item['class']: item['name']} for item in structure_origin_list] # 取得構面字典，型態為list of dict

In [132]:
# 以pattern_text 例如是成就感，那就會去找text中有出現成就感的alias的次數
def get_dict_num(pattern_text, text):
    pattern = r"(" + structure_dict[pattern_text] + ")"
    num = len(re.findall(pattern, text))
    return num

In [161]:
# 取得資料
df_ = pd.read_csv('./interview.csv')
df_.drop(['Unnamed: 0','Unnamed: 0.1','rank','difficult'], axis=1,inplace=True)
df_ = df_[~df_['process'].isna()].reset_index(drop='index') # (7219 , 8)
df_.head()
# df_ = df_[~df_['month_salary'].isna()].reset_index(drop='index')
# df_.drop(['month_salary','year_salary','year-end_salary','bonus','dividends','other','average_day','seniority','total_seniority','mood'], axis = 1, inplace=True)

,company_name,vacancies,post_time,interview_time,status,process,question,suggestion
0,裕隆集團_納智捷汽車股份有限公司,企劃管理師,2022.08.24,2022.07,未錄取,針對專業提出疑問，並...,"[{'Q': '這個品牌業界傳的都不太好，為什麼願意來納智捷面試', 'A': '因為我本來...",NaN
1,裕隆集團_納智捷汽車股份有限公司,管理師,2022.02.21,2019.02,無聲卡,整體面試過程約2小時...,"[{'Q': '對於納智捷的了解', 'A': '依照職位跟公司官網做...'}]",NaN
2,裕隆集團_納智捷汽車股份有限公司,行銷專員,2022.01.13,2017.09,錄取,第一次面試先簡單自我...,"[{'Q': '會不會開車', 'A': '有駕照，但當時還不熟...'}]",NaN
3,裕隆集團_納智捷汽車股份有限公司,人力資源管理師,2021.09.12,2021.09,未錄取,第一關，人資面試\n...,"[{'Q': '結算薪資怎麼做', 'A': '照実回答'}]",NaN
4,裕隆集團_納智捷汽車股份有限公司,秘書,2021.04.21,2017.09,錄取,在台南東區的廠二樓面...,"[{'Q': '與人交談方面的問題', 'A': '我是個開朗的人，比較...'}]",NaN


In [136]:
def isopen(text):
    if("..." not in text):
        return True
    else:
        return False

In [137]:
def get_QA_text(data):
    text = ""
    for item in data:
        text += item['Q'] + " " + item['A'] + " "
    return text

In [162]:
# 只用有解鎖的文章
df_open = df_[df_['process'].apply(lambda x: isopen(x))].reset_index(drop='index')

In [163]:
# 提取question的文章
df_open['Q_content'] = df_open['question'].apply(lambda x: get_QA_text(eval(x)))

In [164]:
df_open.shape

(442, 9)

In [165]:
df_open.head()

,company_name,vacancies,post_time,interview_time,status,process,question,suggestion,Q_content
0,裕隆集團_裕隆汽車製造股份有限公司,客服,2023.03.24,2023.01,等待中,覺得很浪費時間，我面試大夜班，到現場要先測試聽力打字，現場很吵，然後反應也沒用，也不知道在測...,"[{'Q': '沒有問問題', 'A': '基本上都是在講解工作內容'}]",如果你想考車險，他這邊好像會培訓你考證照大概是這樣，我覺得就是普通的客服，薪水就30k差不多...,沒有問問題 基本上都是在講解工作內容
1,裕隆集團_裕隆汽車製造股份有限公司,職員,2023.03.06,2018.01,錄取,主管是一個溫文儒雅的先生 真的感覺得出來他非常誠懇 然後問的問題沒有太多深入的題目 主要就是...,"[{'Q': '論文的撰寫困難', 'A': '照實回答'}]",這家公司印象中是華創 現在也不在了吧 不過基本上裕隆集團真的很讚 到了鬼月還有放假跟旅遊補助...,論文的撰寫困難 照實回答
2,裕隆集團_裕隆汽車製造股份有限公司,物料管理實習生,2023.03.02,2022.05,錄取,面試是以視訊面試進行，除了HR以外還有一個物料管理部門的主管（？）之類的長官，整體面試進行的...,"[{'Q': '自我介紹 3 分鐘', 'A': '主要針對專業領域跟未來規畫進行描述，還有...",可以多了解跟實習部門工作內容相關的時事議題，並有自己的想法。另外，履歷中與部門相關的經歷就算...,自我介紹 3 分鐘 主要針對專業領域跟未來規畫進行描述，還有自己的特質（我是回答喜歡學習），...
3,裕隆集團_裕隆汽車製造股份有限公司,職員,2023.02.16,2021.06,無聲卡,先遠端考英文及專業測驗，大約1個半小時，而後開始視訊面試。面談大約進行一小時，面試官有3-4...,"[{'Q': '簡述碩士論文對該領域的貢獻', 'A': '依實際研究成果回答。'}]",建議可以花時間看看公司官網跟永續報告書、想想為何想到裕隆上班？自身學經歷背景跟應徵職缺的連結...,簡述碩士論文對該領域的貢獻 依實際研究成果回答。
4,裕隆集團_裕隆汽車製造股份有限公司,品質檢驗員,2023.02.10,2022.12,錄取,大門換證帶小黃卡\n人資帶到辦公室，考量測及英文\n第一次接觸品檢很多量測題目真的看不太懂\...,"[{'Q': '為什麼要來我們公司', 'A': '如實回答，不想輪班工作'}]",薪資才3萬多一點，工作算輕鬆，內部主管幾乎都是靠年資上位，主管能力不足，都是靠底下員工溝通解...,為什麼要來我們公司 如實回答，不想輪班工作


In [166]:
# 字典型態的資料
# 資料轉為字典，比較方便去做計算
dict_ = df_open.to_dict(orient='records') 

In [167]:
# 把process、suggestion、Q_content內容合併
for data in dict_:
    data['content'] = str(data['process'])+','+str(data['suggestion'])+','+str(data['Q_content'])

In [168]:
# 取得根據每個構面下的類別名稱，計算該類別名稱出現多少次，來去計算dtm
for structure in list(structure_dict.keys()):
    for data in dict_:
        data[structure] = get_dict_num(structure, data['content'])

In [170]:
# DTM for 趨勢圖
# 把用不到的欄位踢除
kick_off_key = ['company_name','status','process','question','suggestion','frequency_overtime','Q_content']
data_list = [{k: v for k, v in d.items() if k not in kick_off_key} for d in dict_]
# 得到DTM DataFrame
dtm_df = pd.DataFrame(data_list)
dtm_df.shape

(442, 61)

In [171]:
# DTM for correlation & co-ocurence #把用不到的欄位踢除
kick_off_ = ['vacancies','post_time','interview_time','content','company_name','status','process','question','suggestion','frequency_overtime','Q_content']
co_ = [{k: v for k, v in d.items() if k not in kick_off_} for d in dict_]
dtm_co = pd.DataFrame(co_)
dtm_co.head()

,成就感,學習成長,創新,薪資,福利,管理制度,工作氛圍,同事互動,主管風格,工作地點,...,工作,離職,轉職,新鮮人,畢業,出路,能力,招募人員,正向,負向
0,0,0,0,3,1,1,0,0,0,0,...,1,0,0,0,0,0,0,1,2,242
1,0,0,0,0,0,0,1,0,2,0,...,0,0,0,0,0,0,0,2,3,193
2,0,2,0,0,0,5,0,1,2,0,...,2,0,0,0,0,0,4,3,12,654
3,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,2,2,1,192
4,0,0,0,1,0,1,0,1,5,0,...,4,0,0,0,0,0,3,6,2,197


In [172]:
dtm_df.head()

,vacancies,post_time,interview_time,content,成就感,學習成長,創新,薪資,福利,管理制度,...,工作,離職,轉職,新鮮人,畢業,出路,能力,招募人員,正向,負向
0,客服,2023.03.24,2023.01,覺得很浪費時間，我面試大夜班，到現場要先測試聽力打字，現場很吵，然後反應也沒用，也不知道在測...,0,0,0,3,1,1,...,1,0,0,0,0,0,0,1,2,242
1,職員,2023.03.06,2018.01,主管是一個溫文儒雅的先生 真的感覺得出來他非常誠懇 然後問的問題沒有太多深入的題目 主要就是...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,3,193
2,物料管理實習生,2023.03.02,2022.05,面試是以視訊面試進行，除了HR以外還有一個物料管理部門的主管（？）之類的長官，整體面試進行的...,0,2,0,0,0,5,...,2,0,0,0,0,0,4,3,12,654
3,職員,2023.02.16,2021.06,先遠端考英文及專業測驗，大約1個半小時，而後開始視訊面試。面談大約進行一小時，面試官有3-4...,0,0,0,0,0,0,...,0,0,0,0,0,0,2,2,1,192
4,品質檢驗員,2023.02.10,2022.12,大門換證帶小黃卡\n人資帶到辦公室，考量測及英文\n第一次接觸品檢很多量測題目真的看不太懂\...,0,0,0,1,0,1,...,4,0,0,0,0,0,3,6,2,197


### 計算correlation

In [173]:
df_correlation = pd.DataFrame(np.corrcoef(dtm_co.T),columns=dtm_co.columns, index=dtm_co.columns)

c:\Users\User\anaconda3\envs\text_\lib\site-packages\numpy\lib\function_base.py:2691: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
c:\Users\User\anaconda3\envs\text_\lib\site-packages\numpy\lib\function_base.py:2692: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


In [182]:
df_corre_vect = df_correlation.reset_index()
df_corre_vect.rename(columns={'index': 'item1'}, inplace=True)
df_corre_vect.head()

,item1,成就感,學習成長,創新,薪資,福利,管理制度,工作氛圍,同事互動,主管風格,...,工作,離職,轉職,新鮮人,畢業,出路,能力,招募人員,正向,負向
0,成就感,1.000000,0.043843,-0.012927,0.081711,-0.046301,-0.031028,-0.026186,-0.007492,-0.043460,...,0.089708,0.047853,-0.013716,-0.001807,0.014914,-0.039775,-0.034507,-0.038126,0.126932,-0.005064
1,學習成長,0.043843,1.000000,0.060496,-0.040755,0.022032,0.048898,0.008600,0.175008,0.195431,...,0.203451,-0.002673,0.028902,0.252897,0.013700,0.072116,0.202757,0.305837,0.273331,0.328328
2,創新,-0.012927,0.060496,1.000000,-0.028549,-0.029144,0.030081,-0.034661,0.108407,-0.059742,...,-0.001728,-0.022267,-0.006440,-0.017158,-0.014795,-0.018676,0.118290,-0.021991,0.128881,0.044477
3,薪資,0.081711,-0.040755,-0.028549,1.000000,0.584063,0.193192,0.058120,-0.052762,0.002311,...,0.161982,0.152310,-0.030292,0.023903,0.131384,0.013005,-0.002978,0.016203,0.133677,0.170845
4,福利,-0.046301,0.022032,-0.029144,0.584063,1.000000,0.224522,0.074269,-0.050447,-0.033819,...,0.141007,0.041224,-0.030924,0.008752,-0.016902,0.017714,-0.002645,-0.005324,0.222602,0.158065


In [175]:
corr_melt_df = pd.melt(df_corre_vect, id_vars='item1', value_vars=list(df_corre_vect.columns)[1:], var_name='item2', value_name='correlation')
corr_melt_df.rename(columns={'index':'item1'},inplace=True)
corr_melt_df

,item1,item2,correlation
0,成就感,成就感,1.000000
1,學習成長,成就感,0.043843
2,創新,成就感,-0.012927
3,薪資,成就感,0.081711
4,福利,成就感,-0.046301
...,...,...,...
3244,出路,負向,0.022262
3245,能力,負向,0.442509
3246,招募人員,負向,0.624562
3247,正向,負向,0.578160


In [176]:
corr_melt_df[corr_melt_df['item1'] == '負向'].sort_values(['correlation'], ascending=False)[1:7]

,item1,item2,correlation
1025,負向,馬力,0.999949
2564,負向,面試,0.678857
3134,負向,招募人員,0.624562
3191,負向,正向,0.578160
512,負向,主管風格,0.457814
2735,負向,工作,0.454058


### 計算 co-occurrence

In [178]:
result = np.dot(dtm_co.T.values, dtm_co.values)
co_occurrence_df = pd.DataFrame(result, columns=dtm_co.columns, index=dtm_co.columns)
co_occurrence_df.head()

,成就感,學習成長,創新,薪資,福利,管理制度,工作氛圍,同事互動,主管風格,工作地點,...,工作,離職,轉職,新鮮人,畢業,出路,能力,招募人員,正向,負向
成就感,19,9,0,14,1,9,3,4,19,0,...,38,3,0,1,1,0,12,32,77,4169
學習成長,9,338,4,54,51,182,56,105,448,24,...,397,17,2,40,9,22,336,741,790,72241
創新,0,4,6,0,0,6,0,6,1,0,...,6,0,0,0,0,0,16,8,30,1596
薪資,14,54,0,592,243,303,74,32,318,49,...,414,43,0,16,23,16,206,498,716,66516
福利,1,51,0,243,248,213,52,22,187,25,...,259,16,0,9,4,11,136,311,543,42820


In [181]:
df_ocurr_vect = co_occurrence_df.reset_index()
df_ocurr_vect.rename(columns={'index': 'item1'}, inplace=True)
df_ocurr_vect
ocurr_melt_df = pd.melt(df_ocurr_vect, id_vars='item1', value_vars=list(df_ocurr_vect.columns)[1:], var_name='item2', value_name='ocurrelation')
ocurr_melt_df.rename(columns={'index':'item1'},inplace=True)
ocurr_melt_df

,item1,item2,ocurrelation
0,成就感,成就感,19
1,學習成長,成就感,9
2,創新,成就感,0
3,薪資,成就感,14
4,福利,成就感,1
...,...,...,...
3244,出路,負向,10597
3245,能力,負向,221842
3246,招募人員,負向,482379
3247,正向,負向,525814
